In [ ]:
%cd ../src
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
requests.packages.urllib3.disable_warnings() 

import claim_verification

# Apply model on several test claims

### Brad pitt marrying Britney Spears)

In [ ]:
claim = "brad pitt is to marry with britney spears"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

In [ ]:
claim = "brad pitt is not married to britney spears"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

### Joe Biden taking home classified documents

In [ ]:
claim = "Joe Biden took home classified documents after leaving the vice-presidency"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

In [ ]:
claim = "Joe Biden never took home any classified documents after leaving the vice-presidency"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

### COVID-19 vaccine and (in)fertility

In [ ]:
claim = "COVID-19 vaccine causes infertility"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

In [ ]:
claim = "COVID-19 vaccine does not cause infertility"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

### Jan Smit met de US president (fake, but no relevant evidence)

In [ ]:
claim = "jan smit has met with US president in 2012"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

# Apply model on real tweets

### Manual tweet example #1

In [ ]:
tweet = "@ianbremmer Russia also has troops on Moldova's border with Ukraine."
evidence = claim_verification.investigate_claim(tweet, model_type='zero-shot')

In [ ]:
# Experiment: how does rephrasing as a question impact results?
tweet = "does russia have troops on moldova border?"
evidence = claim_verification.investigate_claim(tweet, model_type='zero-shot')

### Filter opinions

In [ ]:
tweet = "Ukraine already is a puppet government. The US started a puppet regime, now the Russians might install their own to secure their border. I feel bad for the civilians caught between them"
evidence = claim_verification.investigate_claim(tweet, model_type='zero-shot')

In [ ]:
tweet = "Ukraine in NATO is how we get to WWIII. I can't see how that ever works when Russia controls Crimea. I think Ukraine would have to agree to give up Crimea and possibly other areas and redraw its eastern border before it could happen. https://t.co/sptiD5yxbi,Ukraine in NATO is how we get to WWIII. I can't see how that ever works when Russia controls Crimea. I think Ukraine would have to agree to give up Crimea and possibly other areas and redraw its eastern border before it could happen."
evidence = claim_verification.investigate_claim(tweet, model_type='zero-shot')

### Questionable news from Russian sources

Example of questionable news by Russian press service and media channels. This news was denied by Ukraine. Note: Russian news sources might be filtered from search results in Poland.

In [ ]:
tweet = "The Russian military destroyed two infantry fighting vehicles of the Ukrainian army, which crossed the Russian border from Ukraine for the emergency evacuation of the Ukrainian sabotage group, reports the Southern Military District."
evidence = claim_verification.investigate_claim(tweet, model_type='zero-shot')

### Breitbart news article

In [ ]:
news = 'WWIII Watch: E.U. Warns China Arming Russia Would Cross ‘Red Line’'
evidence = claim_verification.investigate_claim(news, model_type='zero-shot')